In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

In [ ]:
df = pd.read_csv("MGP2025.csv")
df.head(3)

In [ ]:

# === 2️⃣ Create a DNF flag and clean main race finish ===
df["DNF_Flag"] = (df["MainRaceFinish"] == 23).astype(int)
df["MainRaceFinishClean"] = df["MainRaceFinish"].replace(23, np.nan)

# Drop rows with missing GridPosition or SprintFinish
df = df.dropna(subset=["GridPosition", "SprintFinish"])

# === 3️⃣ Define features for both models ===
feature_cols = ["RiderName", "GridPosition", "SprintFinish"]

# === 4️⃣ Preprocessing ===
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["RiderName"]),
        ("num", StandardScaler(), ["GridPosition", "SprintFinish"])
    ]
)



In [ ]:
X_dnf = df[feature_cols]
y_dnf = df["DNF_Flag"]

dnf_classifier = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(n_estimators=300, random_state=42))
])

X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_dnf, y_dnf, test_size=0.2, random_state=42)
dnf_classifier.fit(X_train_d, y_train_d)

y_pred_d = dnf_classifier.predict(X_test_d)
dnf_acc = accuracy_score(y_test_d, y_pred_d)
print(f"🏁 DNF Classifier Accuracy: {dnf_acc:.3f}")

In [ ]:
df_no_dnf = df[(df["DNF_Flag"] == 0) & (df["MainRaceFinishClean"].notna())]

X_finish = df_no_dnf[feature_cols]
y_finish = df_no_dnf["MainRaceFinishClean"]

regressor = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_estimators=300, random_state=42))
])

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_finish, y_finish, test_size=0.2, random_state=42)
regressor.fit(X_train_f, y_train_f)

y_pred_f = regressor.predict(X_test_f)
mae = mean_absolute_error(y_test_f, y_pred_f)
rmse = np.sqrt(mean_squared_error(y_test_f, y_pred_f))

print(f"🏎️  Regressor Performance: MAE={mae:.2f}, RMSE={rmse:.2f}")

In [ ]:
new_data = pd.DataFrame({
    "RiderName": ["Pedro Acosta"],  # 👈 change rider name here
    "GridPosition": [2],                 # 👈 change grid position here
    "SprintFinish": [2]                  # 👈 change sprint finish result here
})

In [ ]:
# Ensure new_data has same dtypes as training features
import math
new_data = new_data.astype(X_dnf.dtypes.to_dict())

dnf_prob = dnf_classifier.predict_proba(new_data)[0, 1]

if dnf_prob > 0.5:
    print("\n🔮 Prediction: Likely DNF (High probability)")
else:
    finish_pred = regressor.predict(new_data)[0]
    finish_pred = max(1, math.floor(finish_pred))  # floor + avoid 0
    print(f"\n🔮 Predicted Finish Position: {finish_pred}")
print(f"Chance of DNF: {dnf_prob*100:.2f}%")